# VacationPy 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from geo_keys import g_key

# Store part I results into DataFrame

-Load the csv exported in part 1 to a DataFrame

In [2]:
city_weather=pd.read_csv('./Resources/weather_csv')
city_weather.dropna(inplace=True)
city_weather.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,khatanga,71.9667,102.5000,3.58,97.0,88.0,3.33,RU,1.666394e+09
1,constitucion,-35.3333,-72.4167,53.92,74.0,1.0,14.14,CL,1.666394e+09
2,hobart,-42.8794,147.3294,64.26,88.0,75.0,4.61,AU,1.666394e+09
3,ardakan,32.3100,54.0175,68.50,19.0,0.0,3.11,IR,1.666394e+09
4,albany,42.6001,-73.9662,52.45,52.0,3.0,3.69,US,1.666394e+09


Humidity Heatmap 

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_weather[["Lat", "Lng"]]
humidity=city_weather["Humidity"]


In [5]:
fig=gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=97,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create a new DataFrame fitting weather criteria 

In [6]:
#Narrow down the cities to fit weather conditions
#Drop any rows with null values 

In [7]:
vacation_weather=city_weather
vacation_weather=vacation_weather.loc[(vacation_weather["Max Temp"] <80) & (vacation_weather["Max Temp"] >70)]
vacation_weather=vacation_weather.loc[(vacation_weather["Wind Speed"] <10)]
vacation_weather=vacation_weather.loc[(vacation_weather["Cloudiness"] ==0)]

In [8]:
vacation_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,bilma,18.6853,12.9164,74.71,22.0,0.0,9.51,NE,1.666394e+09
62,cayenne,4.9333,-52.3333,78.84,89.0,0.0,0.00,GF,1.666394e+09
84,oconomowoc,43.1117,-88.4993,71.17,41.0,0.0,6.91,US,1.666394e+09
98,chokwe,-24.5333,32.9833,71.83,80.0,0.0,4.50,MZ,1.666394e+09
219,patiya,22.2931,91.9762,73.02,91.0,0.0,3.09,BD,1.666394e+09
292,sann,26.0418,68.1378,78.44,37.0,0.0,7.05,PK,1.666394e+09
306,mirpur mathelo,28.0227,69.5489,74.44,37.0,0.0,3.51,PK,1.666394e+09
360,riyadh,24.6877,46.7219,71.74,16.0,0.0,6.29,SA,1.666394e+09
409,worland,44.0169,-107.9554,71.89,21.0,0.0,5.99,US,1.666394e+09
515,charters towers,-20.1000,146.2667,78.69,58.0,0.0,2.80,AU,1.666394e+09


# Hotel Map

In [9]:
hotels=[]

hotel_df=vacation_weather
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



for index, row in hotel_df.iterrows():
    city=row["City"]
    lat=row["Lat"]
    lng=row["Lng"]
    
    
    params= {
    "location": f"{lat}, {lng}",
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
    
    
    
    #assemble url and make API request
    request=requests.get(base_url, params=params)
    response=request.json()
    
    #extract results
    hotel_results=response['results']
    
    try:
       
        hotels.append(response["results"][0]["name"])
    except:
        print("No hotel found nearby")
        
# Add 'Hotel Name' column to the dataframe

hotel_df["Hotel Name"]=hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,bilma,18.6853,12.9164,74.71,22.0,0.0,9.51,NE,1.666394e+09,Expeditions Ténére voyages
62,cayenne,4.9333,-52.3333,78.84,89.0,0.0,0.00,GF,1.666394e+09,Hôtel Le Dronmi
84,oconomowoc,43.1117,-88.4993,71.17,41.0,0.0,6.91,US,1.666394e+09,Hilton Garden Inn Oconomowoc
98,chokwe,-24.5333,32.9833,71.83,80.0,0.0,4.50,MZ,1.666394e+09,Complexo Sonho Real
219,patiya,22.2931,91.9762,73.02,91.0,0.0,3.09,BD,1.666394e+09,Shil para
292,sann,26.0418,68.1378,78.44,37.0,0.0,7.05,PK,1.666394e+09,New Sindhri Hotel
306,mirpur mathelo,28.0227,69.5489,74.44,37.0,0.0,3.51,PK,1.666394e+09,Leopards courier mirpur mathelo
360,riyadh,24.6877,46.7219,71.74,16.0,0.0,6.29,SA,1.666394e+09,"InterContinental Riyadh, an IHG Hotel"
409,worland,44.0169,-107.9554,71.89,21.0,0.0,5.99,US,1.666394e+09,Town House Motor Inn
515,charters towers,-20.1000,146.2667,78.69,58.0,0.0,2.80,AU,1.666394e+09,Cattleman's Rest Motor Inn


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers= gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))